In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import clip
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision.models import resnet18
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from scipy.optimize import linear_sum_assignment

from data import get_bloodmnist_loader

!nvidia-smi

Sun Nov 24 23:09:07 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:D8:00.0 Off |                    0 |
| N/A   34C    P8    15W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
train_loader = get_bloodmnist_loader(split='train', batch_size=64)
val_loader = get_bloodmnist_loader(split='val', batch_size=100)

Using downloaded and verified file: /h/lkcai/.medmnist/bloodmnist_128.npz
Using downloaded and verified file: /h/lkcai/.medmnist/bloodmnist_128.npz


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

bloodmnist_label = {
    "0": "basophil",
    "1": "eosinophil",
    "2": "erythroblast",
    "3": "immature granulocytes(myelocytes, metamyelocytes and promyelocytes)",
    "4": "lymphocyte",
    "5": "monocyte",
    "6": "neutrophil",
    "7": "platelet",
}


model, preprocess = clip.load("RN50", device=device)
texts = clip.tokenize(["a photo of " + v for v in bloodmnist_label.values()]).to(device)

correct = 0
total = 0

with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images = images.to(device)
        labels = labels.to(device).view(-1)
            
        logits_per_image, logits_per_text = model(images, texts)
        probs = logits_per_image.softmax(dim=-1).max(1)[1]
        
        correct += probs.eq(labels).sum().item()
        total += labels.size(0)

accuracy = 100. * correct / total
print(accuracy)

100%|██████████| 18/18 [00:07<00:00,  2.26it/s]

24.299065420560748
